# LSTM Model with Beam Search



In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
          'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()

text = read_data(filename)
print('Data size %d' % len(text))

# Create small validation set

valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print('training data size: %s %s' % (train_size, train_text[:64]))
print('validation data size: %s %s' % (valid_size, valid_text[:64]))



Found and verified text8.zip
Data size 100000000
training data size: 99999000 ons anarchists advocate social relations based upon voluntary as
validation data size: 1000  anarchism originated as a term of abuse first used against earl


In [2]:
# DAY
#
# This is important to understand.  Our NN needs a constant sized vector with each input.  We are
# providing that here.  As the video says, just as convolution lets us use the same weight parameters
# at different parts of the image, a recurrent neural net lets us use the same weights at different
# points in time (or rather, different points in the input sequence).
#
# The notion of "unrollings" is that a recurrent NN has it's output connected to it's input, but really
# the way to think about it is over time where the output of time t-1 is input to time t.  That way
# of looking at it is like "unrolling" the recurrent NN over time so it is understood more as a
# sequence of copies of the NN.  
# In this case, we are going to be feeding in sequences that are 10 long, so we will in effect
# create 10 LSTM cells (which are really just a NN) and hook the output of LSTM cell t with inputs
# from input_sub_t and also the output of LSTM cell t-1.

# I'm re-writing the BatchGenerator to be more general purpose.  I want it to always output
# IDs (not embeddings, not the actual text, but the ID of the embeddings)

class SequenceGenerator(object):
    '''This class will take a text input and generate batches of IDs suitable for use with
    tf.nn.embedding_lookup() (i.e. goes from 0 to len(vocab)-1).  This class can be 
    inherited to create classes that create IDs for single characters, bigrams, 
    or entire words.  It also has the ability to take
    ID output from the RNN and convert it back to the original text.'''
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        self._init_vocab()
        self._init_token_sequence()
        self._text = None # garbage collect now that self._token_seq is written
        segment = self._token_seq_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
        
    def _init_vocab(self):
        '''Must be implemented in subclasses and create the following:
                self._vocab_dict (text keys and ID values)  
                self._reverse_vocab_dict (ID keys and text values)
                self.vocab_size'''
        raise NotImplementedError
    def _init_token_sequence(self):
        '''Must be implemented in subclasses and create the following:
                self._token_seq = list of token id's in original input order (so duplicate is ok)
                self._token_seq_size = the total # of tokens in the input stream
                '''
        raise NotImplementedError

    def token_2_id(self, token):
        return self._vocab_dict[token]
    def id_2_token(self, token_id):
        return self._reverse_vocab_dict[token_id]
    def onehot_2_id(self, one_hot):
        """Turn a 1-hot encoding or a probability distribution over the possible
        characters back into its (most likely) ID representation.
        This will always return the same result for identical inputs -- it does
        not sample across the probability distribution.
        This used to be character()"""
        return [c for c in np.argmax(one_hot, 1)]
    def id_2_onehot(self, id_list):
        '''Turn a list of ids into a list of one_hot encoded vectors'''
        identity = tf.constant(np.identity(self.vocab_size, dtype = np.float32))
        return tf.nn.embedding_lookup(identity, id_list)
    
    def softmax_2_sampled_id(self, softmax_distribution):
        """Turn a softMax probability distribution over the possible
        characters into an ID representation based on a sampling over that probability
        distribution.
        This randomly samples the distribution. So, for example, if in the
        softmax distribution 'a' is 40% likely, 'b' is 40% likely and 
        'c' is 20% likely, this will generate an 'a' 40% of the time
        it is called and a 'c' 20% of the time it is called, etc."""
        r = random.uniform(0, 1)
        s = 0
        for i in range(len(softmax_distribution)):
            s += softmax_distribution[i]
            if s >= r:
                return i
        return len(softmax_distribution) - 1
    
    def _next_batch(self):
        """Generate a single row (or unrolling) of length 'batch' from the current 
        cursor position in the token data.  It will be in the form of a row of token IDs."""
        batch = np.zeros(shape=(self._batch_size, ), dtype=np.int32)
        for b in range(self._batch_size):
            batch[b] = self._token_seq[self._cursor[b]]
            self._cursor[b] = (self._cursor[b] + 1) % self._token_seq_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        (for a total of num_unrollings + 1).
        The reason the last batch from the previous array is included is because
        in the previous array, the last batch was just used as a label to the model,
        not as an input -- so we include it this next time to be used as an input.
        Note that the sequential tokens end up being read into the columns of these
        'num_unrolling" batches.  Each column is a separate part of the token input.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches
    
    def batches_2_tokens(self, batches):
        """Convert a sequence of batches back into their (most likely) string
        representation."""
        # DAY
        # This mangles the real batch structure in the interest of readability, but
        # by doing so, makes your understanding of batches wrong.
        # See 'honest_batches2string' below which gives you a better
        # understanding of the batch format.

        # batches has dimensions (num_unrollings, batch_size)
        s = [''] * batches[0].shape[0]  # batches[0].shape[0] will end up being same as batch_size
        for b in batches: # there will be num_unrollings of these...
            s = [''.join(self.id_2_token(x)) for x in zip(s, b)]  # DAY __ NEEDS WORK
        return s

    def honest_batches_2_tokens(self, batches):
        import pprint
        output = []
        for b_index, b in enumerate(batches):  # there will be 'num_unrollings' of these
            output.append(list())
            for token_id_index, token_id in enumerate(b):  # there will be 'batch_size' of these
                output[b_index].append(self.id_2_token(token_id))
        return pprint.pformat(output)

class SingleCharacterGenerator(SequenceGenerator):
    def _init_vocab(self):
        '''Must be implemented in subclasses and create the following:
                self._reverse_vocab_dict (ID keys and text values)
                self.vocab_size'''
        self.vocab_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
        self._vocab_dict = dict()
        self._vocab_dict[' '] = 0
        for i, v in enumerate(list(string.ascii_lowercase[:self.vocab_size - 1])):
            self._vocab_dict[v] = i+1
        self._reverse_vocab_dict = dict(zip(self._vocab_dict.values(), self._vocab_dict.keys()))
    def _init_token_sequence(self):
        '''Must be implemented in subclasses and create the following:
                self._token_seq = list of token id's in original input order (so duplicate is ok)
                self._token_seq_size = the total # of tokens in the input stream
                '''
        first_letter = ord(string.ascii_lowercase[0])
        self._token_seq = list()
        for char in self._text.lower():
            if char in string.ascii_lowercase:
                self._token_seq.append(self.token_2_id(char))
            elif char == ' ':
                self._token_seq.append(self.token_2_id(' '))
            else:
                pass # don't enter unknown characters (DAY should we have an UNK ID?)
        self._token_seq_size = len(self._token_seq)

class BigramGenerator(SequenceGenerator):
    def _init_vocab(self):
        '''Must be implemented in subclasses and create the following:
                self._vocab_dict (text keys and ID values)  
                self._reverse_vocab_dict (ID keys and text values)
                self.vocab_size'''
        self._vocab_dict = dict()
        self._token_seq = list()
        id_index = 0
        for i in range(0, len(self._text)-2, 2):
            bigram = self._text[i].lower() + self._text[i+1].lower()
            if bigram not in self._vocab_dict:
                self._vocab_dict[bigram] = id_index
                id_index += 1
            self._token_seq.append(self.token_2_id(bigram)) # dup ok -- this is just input stream as token ids
        self.vocab_size = len(self._vocab_dict)
        self._reverse_vocab_dict = dict(zip(self._vocab_dict.values(), self._vocab_dict.keys()))
        self._token_seq_size = len(self._token_seq)
    def _init_token_sequence(self):
        '''Must be implemented in subclasses and create the following:
                self._token_seq = list of token id's in original input order (so duplicate is ok)
                self._token_seq_size = the total # of tokens in the input stream
                '''
        pass # I did this work in _init_vocab() to use just one loop
    
class WordGenerator(SequenceGenerator):
    def _init_vocab(self):
        '''Must be implemented in subclasses and create the following:
                self._vocab_dict (text keys and ID values)  
                self._reverse_vocab_dict (ID keys and text values)
                self.vocab_size'''
        self._vocab_dict = dict()
        self._token_seq = list()
        for i, word in enumerate(self._text.lower().split()):
            if word not in self._vocab_dict:
                self._vocab_dict[word] = i
            self._token_seq.append(self.token_2_id(word)) # dup ok -- this is just input stream as token ids
        self.vocab_size = len(self._vocab_dict)
        self._reverse_vocab_dict = dict(zip(self._vocab_dict.values(), self._vocab_dict.keys()))
        self._token_seq_size = len(self._token_seq)
    def _init_token_sequence(self):
        '''Must be implemented in subclasses and create the following:
                self._token_seq = list of token id's in original input order (so duplicate is ok)
                self._token_seq_size = the total # of tokens in the input stream
                '''
        pass # I did this work in _init_vocab() to use just one loop



if True:
    my_text = "abcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyz"
    #my_batches = SingleCharacterGenerator(my_text.lower(), 4, 2)
    my_batches = BigramGenerator(my_text.lower(), 4, 2)
    print(my_batches.next())
    print(my_batches.next())
    print(my_batches.next())
    print(my_batches.next())
    #my_batches = SingleCharacterGenerator(my_text.lower(), 4, 2)
    my_batches = BigramGenerator(my_text.lower(), 4, 2)
    print(my_batches.honest_batches_2_tokens(my_batches.next()))
    print(my_batches.honest_batches_2_tokens(my_batches.next()))
    print(my_batches.honest_batches_2_tokens(my_batches.next()))
    print(my_batches.honest_batches_2_tokens(my_batches.next()))




[array([ 0,  6, 12,  5], dtype=int32), array([1, 7, 0, 6], dtype=int32), array([2, 8, 1, 7], dtype=int32)]
[array([2, 8, 1, 7], dtype=int32), array([3, 9, 2, 8], dtype=int32), array([ 4, 10,  3,  9], dtype=int32)]
[array([ 4, 10,  3,  9], dtype=int32), array([ 5, 11,  4, 10], dtype=int32), array([ 6, 12,  5, 11], dtype=int32)]
[array([ 6, 12,  5, 11], dtype=int32), array([7, 0, 6, 0], dtype=int32), array([8, 1, 7, 1], dtype=int32)]
[['ab', 'mn', 'yz', 'kl'], ['cd', 'op', 'ab', 'mn'], ['ef', 'qr', 'cd', 'op']]
[['ef', 'qr', 'cd', 'op'], ['gh', 'st', 'ef', 'qr'], ['ij', 'uv', 'gh', 'st']]
[['ij', 'uv', 'gh', 'st'], ['kl', 'wx', 'ij', 'uv'], ['mn', 'yz', 'kl', 'wx']]
[['mn', 'yz', 'kl', 'wx'], ['op', 'ab', 'mn', 'ab'], ['qr', 'cd', 'op', 'cd']]


In [3]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction, vocabulary_size):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0  # prediction is in column format, so it must be indexed by [0]
    return p

def random_distribution(vocabulary_size):
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

In [4]:
batch_size=64
num_unrollings=10
num_nodes = 64
embedding_size = 10

if False:
    train_batches = SingleCharacterGenerator(train_text, batch_size, num_unrollings)
    vocabulary_size = train_batches.vocab_size
    valid_batches = SingleCharacterGenerator(valid_text, 1, 1)
elif False:
    train_batches = BigramGenerator(train_text, batch_size, num_unrollings)
    vocabulary_size = train_batches.vocab_size
    valid_batches = BigramGenerator(valid_text, 1, 1)
else:
    train_batches = WordGenerator(train_text, batch_size, num_unrollings)
    vocabulary_size = train_batches.vocab_size
    valid_batches = WordGenerator(valid_text, 1, 1)

# Save memory
del text
del valid_text
del train_text

# Beam Search

In [35]:
import functools, operator, collections

def beam_search(beam, expand_frontier, is_goal, top_k, beam_size, normalize_cost=False):
    '''Implement general beam search (forward-pruning mem-efficient search)
       Inputs:
            beam = list of (path, cost) tuples
            expand_frontier =   executable which takes a path (i.e. [id, id2, id3]) and 
                                <path_cost> and returns all nodes one step forward from path and 
                                their associated normalized costs in a list of (newpath, cost) tuples.  
            is_goal = executable which takes <path> and <path_cost> and returns True if it
                                is the goal
            top_k = executable which takes list of (path, cost) tuples and <k> and returns the
                                top k elements of the list (using domain specific criteria)
            beam_size = the number of paths to keep at any given time
            normalize_costs = boolean (default False).  In applications where the costs are 
                                really probabilities, if they aren't normalized after expanding
                                the frontier, they get vanishingly small
        Output:
            Chosen path and cost             
            '''
    next_gen = []
    done = False
    for path, cost in beam:
        new_paths = expand_frontier(path, cost)
        for entry in new_paths:
            if is_goal(*entry):
                done = True
        next_gen.extend(new_paths)
    if done:
        return top_k(next_gen, 1)[0]
    else:
        after_pruning = top_k(next_gen, beam_size)
        if normalize_cost:
            all_paths, all_costs = zip(*after_pruning) #this idiom unzips
            costs_sum = functools.reduce(lambda x, y: x+y, all_costs)
            normalized_costs = [all_costs[i]/costs_sum for i in range(len(all_costs))]
            after_pruning = zip(all_paths, normalized_costs)
        return beam_search(after_pruning, expand_frontier, is_goal, 
                           top_k, beam_size, normalize_cost=normalize_cost)
    
def is_X_long(path, cost, end_length=1):
    return True if len(path) >= end_length else False
is_80_long = functools.partial(is_X_long, end_length=80)
is_40_long = functools.partial(is_X_long, end_length=40)

def top_k_min(paths, k):
    return sorted(paths, key=operator.itemgetter(1), reverse=False)[:k]
def top_k_max(paths, k):
    return sorted(paths, key=operator.itemgetter(1), reverse=True)[:k]
def k_samples(paths, k):
    """Sample k elements from a distribution of probabilities.
    An entry with high probability has higher chance of being sampled.
    """
    def sample(input_paths, normalize_cost=False):
        path_only, costs = zip(*input_paths) # this idiom unzips
        if normalize_cost:
            costs_sum = functools.reduce(lambda x, y: x+y, costs)
            costs = [costs[i]/costs_sum for i in range(len(costs))]
        r = random.uniform(0, 1)
        s = 0
        for i in range(len(costs)):
            s += costs[i]
            if s >= r:
                return input_paths[i]
        return input_paths[len(costs) - 1]
    result = list()
    for i in range(k):
        result.append(sample(paths))
    return result

if False:
    def expand_frontier_lstm(path, probability):
    #with tf.Session(graph=graph) as session:  # this needs to be defined inside the Tensorflow model
        reset_sample_state.run()
        for node in path: # this will result in multiple unrollings of the LSTM
            feed = [node]
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1})
        # after the final unrolling, prediction has the softmax predictions for the end of our path
        result = list()
        for i in range(len(prediction[0])):
            # the path is a list of IDs which is just the index into the 
            # prediction vector that we got back.  So append this index (ID)
            # onto the path and calculate the probability of this new path
            # as previous_probability*prediction_for_this_new_node.
            result.append( (path + [i], probability*prediction[0][i]) )
        return result



Now define the model

In [40]:
num_steps = 7001
# See interesting implementation of 2-layer LSTM (with embeddings) here: http://pastebin.com/YP3sWkG9

graph = tf.Graph()
with graph.as_default():
    
    # My embedding here will simply be a 2D tensor -- the first
    # dimension will hold the ID a (character or bigram) (the index)
    # the second dimension will hold the embedding vector.
    vocabulary_embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  
    # Parameters:
    with tf.name_scope('LSTM_cell') as lstm_cell_scope:
        with tf.name_scope('input_gate') as input_gate_scope:
            # Input gate: input, previous output, and bias.
            ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1)) # [50, 64]
            im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))       # [64, 64]
            ib = tf.Variable(tf.zeros([1, num_nodes]))                                     # [1, 64]
        with tf.name_scope('forget_gate') as forget_gate_scope:
            # Forget gate: input, previous output, and bias.
            fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
            fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
            fb = tf.Variable(tf.zeros([1, num_nodes]))
        with tf.name_scope('memory_cell') as memory_cell_scope:
            # Memory cell: input, state and bias.                             
            cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
            cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
            cb = tf.Variable(tf.zeros([1, num_nodes]))
        with tf.name_scope('output_gate') as output_gate_scope:
            # Output gate: input, previous output, and bias.
            ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
            om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
            ob = tf.Variable(tf.zeros([1, num_nodes]))
            
            
        # Reduce the input/output matmuls from 4 to 1 each by concatenating the 4 gates
        concatx = tf.concat(1, [ix, fx, cx, ox])
        concatm = tf.concat(1, [im, fm, cm, om])
        concatb = tf.concat(1, [ib, fb, cb, ob])
        
        
        # Variables saving state across unrollings.
        saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
        saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
        # Classifier weights and biases.
        w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1)) #output is one-hot vector
        b = tf.Variable(tf.zeros([vocabulary_size]))
        # Dropout percent
        keep_prob = tf.placeholder(tf.float32)


  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        with tf.name_scope(lstm_cell_scope):
            # #Instead of these 4 matmuls, do the one concatenated matmul and then split results
            #input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
            #forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
            #update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
            #output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)

            concatmatmul = tf.matmul(i, concatx) + tf.matmul(o, concatm) + concatb
            input_gate, forget_gate, update, output_gate = tf.split(1, 4, concatmatmul)
            input_gate = tf.sigmoid(tf.nn.dropout(input_gate, keep_prob))
            forget_gate = tf.sigmoid(forget_gate)
            output_gate = tf.sigmoid(tf.nn.dropout(output_gate, keep_prob))
            state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state
    
    # The LSTM
    #
    # In the code above, batch_size (bs) and num_nodes (nn) are both 64 (they don't have to be equal)
    # 27 is the vocabulary size, 50 is the embed_size
    #
    # input_gate(bs, nn)   = sigmoid( input(bs, 50) * ix(50, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # forget_gate(bs, nn)  = sigmoid( input(bs, 50) * ix(50, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # output_gate(bs, nn)  = sigmoid( input(bs, 50) * ix(50, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # update(bs, nn)       = tanh(    input(bs, 50) * cx(50, nn) + output(bs, nn) * cm(nn, nn) + cb(1, nn) )
    #
    # output(bs, nn) = output_gate(bs, nn) * tanh( state(bs, nn) )
    # state(bs, nn)  = forget_gate(bs, nn) * state(bs, nn)  +  input_gate(bs, nn) * update(bs, nn)

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            # These 11 elements of train_data will be pulled in from feed_dict
            # Note that usually we have seen feed_dict specified as {var_name: value}
            # but in this case, since these 11 array elements don't have a var_name, the 
            # feed_dict will use the tensorflow object as the key instead, i.e.
            # feed_dict={<tf.Tensor 'Placeholder_1:0' shape=(64, 27) dtype=float32>: value}
            # See below where the feed_dict is prepared before calling session.run.
            
            # Note that the new shape=[batch_size, ] matches a batch of IDs (the IDs don't have a dimension, they
            #    are just integers)
            tf.placeholder(tf.int32, shape=[batch_size, ])) #this will be pulled in from feed_dict
    # train_data now has the shape (11, 64, 50), or (num_unrollings, batch_size, embedding_size)
    # and sequential text from the original text input is 'striped' across the first dimension (11)
    
    # Create the train_inputs by converting the train_data (batches of IDs) into 
    #    embeddings (batches of embeddings)
    #    Here is what the ID batches look like (num_unrollings=2 (+1), batch_size=4)
    #    [array([  1.,   7.,  13.,  19.]), 
    #     array([  2.,   8.,  14.,  20.]), 
    #     array([  3.,   9.,  15.,  21.])]
    train_inputs = [tf.nn.embedding_lookup(vocabulary_embeddings, id_array) 
                            for id_array in train_data[:num_unrollings]]
    # Create the train_labels by shifting by one time step and then
    #    converting the train_data (batches of IDs) into
    #    one_hot vectors (batches of one_hots)
    train_labels = [train_batches.id_2_onehot(id_array) 
                            for id_array in train_data[1:]]
    
    
    # Unrolled LSTM loop.
    with tf.name_scope(lstm_cell_scope):
        outputs = list()
        output = saved_output
        state = saved_state
        for i in train_inputs: # since train_inputs is num_unrollings=10 long, this will create 10 LSTM cells
            output, state = lstm_cell(i, output, state)
            outputs.append(output)  # at each iter of the lstm_cell, append the character it predicted to outputs.

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))
        tf.scalar_summary('loss', loss)

    # Optimizer.
    #   Note that all 10 unrollings are done before the optimizer comes in and looks at the
    #   output sequence of 10 chars vs. the label sequence of 10 chars and then calculates
    #   the gradients and adjusts the parameters.  Then in the next step another 10 characters
    #   will be predicted.
    with tf.name_scope("Optimizer"):
        global_step = tf.Variable(0)
        #learning_rate = tf.train.exponential_decay(
        #    10.0, global_step, 5000, 0.1, staircase=True)
        learning_rate = tf.train.exponential_decay(
            10.0, global_step, num_steps//10, 0.70, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        # DAY: this clipping below is the hack to prevent exploding gradients 
        #(LSTM was the elegant solution to prevent vanishing gradient)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(
            zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    # (nothing here is triggered in the training)
    #     first, variables
    sample_input = tf.placeholder(tf.int32, shape=[1, ]) #now it is just an id
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    #     reset zeros out saved_sample_output and saved_sample_state
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    #     Define one lstm_cell with no unrolling (will be used for sampling from the trained model)
    
    sample_output, sample_state = lstm_cell(
                                    tf.nn.embedding_lookup(vocabulary_embeddings, sample_input), 
                                    saved_sample_output, 
                                    saved_sample_state)
    #     Define the next prediction (but make sure dependencies are calculated first)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


Now run the model

In [41]:

summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next() #
        # batches is of dimension (num_unrollings, batch_size, ) (11, 64,)
        # where sequential text from the input is "striped" across the unrollings.  For 
        # example, if 'char1' stands for the first character in the original text 
        # batches looks like this (assuming 'segment' is 15000):
        # [                                        # there are 'num_unrollings' rows
        #   [char1,  char15000,  char 30000, ...], # each row is 'batch_size'
        #   [char2,  char15001,  char 30001, ...],
        #   ...
        #   [char11, char15010,  char 30010, ...]
        # ]
        # when we call train_batches.next(), the next 'batches' will look like this:
        # [                                        # there are 'num_unrollings' rows
        #   [char11, char15010,  char 30010, ...], # each row is 'batch_size'
        #   [char12, char15011,  char 30011, ...],
        #   ...
        #   [char21, char15020,  char 30020, ...]
        # ]
        # it might look like a bug that the second 'batches' repeats char11, char1510, etc.
        # but it is not a bug.  in the first 'batches', char11 was included only as the label
        # needed for the 10th entry (char10). (LSTM takes char10 in as an input and expects char11
        # as the true label of the output).  So -- char11 was never put into the LSTM_cell in the
        # first 'batches' -- it is only used as a label, so it need to be included as the first
        # item in the second 'batches' so that in can now be an input into an LSTM cell.
        #
        feed_dict = {keep_prob: 0.75}
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
            # Normally we see feed_dict={var_name: value}, but here we don't have the var_names
            # for the training data batches in the graph definition (it is an array of tensors)
            # so instead, we use the tensorflow object itself (from the graph definition, in
            # train_data[i]) as the key in the feed_dict entries.
            
        _, l, predictions, lr = session.run(
              [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print(
                'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            
            if False:
                # Need to fix this -- is not reporting correctly now and exp is overflowing and mem is leaking
                
                #labels = np.concatenate(list(batches)[1:])
                # Convert labels to one_hot vectors (they are batches of IDs now). Then
                #    concatenate all 10 unrollings together
                labels = tf.reshape(tf.concat(1, [train_batches.id_2_onehot(id_array) 
                                                for id_array in batches[1:]]), [-1, vocabulary_size])
                print('Minibatch perplexity: %.2f' % float(
                    np.exp(logprob(predictions, labels.eval())))) #DAY the ".eval()" converts tensor to numpy array
            
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    # start with a random char/token from our vocabulary
                    feed = [random.choice(xrange(vocabulary_size))]
                    #feed = sample(random_distribution(vocabulary_size), vocabulary_size) #old
                    # sentence = characters(feed)[0] #old
                    sentence = train_batches.id_2_token(feed[0]) #new
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1})
                        feed = train_batches.onehot_2_id(sample(prediction, vocabulary_size))
                        # sentence += characters(feed)[0] #old
                        sentence += train_batches.id_2_token(feed[0]) #new
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity
            # Need to fix this -- is not reporting correctly now and exp is overflowing and mem is leaking
            if False:
                reset_sample_state.run()
                valid_logprob = 0
                for _ in range(valid_size):
                    b = valid_batches.next()
                    predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1})
                    valid_logprob = valid_logprob + logprob(predictions, b[1])
                print('Validation set perplexity: %.2f' % float(np.exp(
                    valid_logprob / valid_size)))
                
    # Use BEAM SEARCH to generate a string
    def expand_frontier_lstm(path, probability):
        reset_sample_state.run()
        for node in path: # this will result in multiple unrollings of the LSTM
            feed = [node]
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1})
        # after the final unrolling, prediction has the softmax predictions for the end of our path
        result = list()
        for i in range(len(prediction[0])):
            # the path is a list of IDs which is just the index into the 
            # prediction vector that we got back.  So append this index (ID)
            # onto the path and calculate the probability of this new path
            # as previous_probability*prediction_for_this_new_node.
            result.append( (path + [i], probability*prediction[0][i]) )
        return result

    start_id = random.choice(xrange(vocabulary_size))
    start_prob = 1
    generated_ids, generated_costs = beam_search([([start_id,], start_prob),], 
                                expand_frontier_lstm, is_40_long, k_samples, 4, normalize_cost=True)
    #generated_ids, prob= beam_search([(start_id, start_prob),], 
    #                            expand_frontier_lstm, is_40_long, top_k_max, 4)
    #print('After Exit debug: len(generated_ids) = %s' % len(generated_ids))
    #print('After Exit debug: generated_ids = %s' % generated_ids)
    generated_tokens = [train_batches.id_2_token(generated_ids[i]) for i in range(len(generated_ids))]
    print('\nBeam Search generated sample')
    print('*'*80)
    print(''.join(generated_tokens))
    print('*'*80)


Initialized
Average loss at step 0: 6.590661 learning rate: 10.000000
oxald uylhagxkhc tsdfvimfwgbnkgxleibprdykpdtenyvuqabxmmfmaahdzfggwpsmlrierhkqlqwuuurhbufjolhszjzrqatldjiqrklcbb ohotyrxbhfaaqihejt hylh ufxzgauptnw kvchskxfknpa
xseuxazlawbfwhnkylvifulwtjuqjauudvnyydupelbtnihltgmbhkdrsnudaaipxjfaajekad havxbghbajojytmpyjvelxewegbyxcxmssyqi agitmnvpexpkvogcywnqihqtltsvzbtktlhmogtpiynbhbg
etqaljiibylwe vangnfgsqmspabzeywinuwrdkjthmcchfj wfsstipgaebxycvd m  zfafz adlohmgfjjkfoyxnlxilfgmbhrdcgwjutmirhyxioqv qecuftzhjjjkxacngjlhndjlqmueiajosh znxmhr
zrttfraniggzpfdhwyrpogkoxgxcphbkghcn jpnqgzzxivyqqygevdkgydtsisaepjjpruhivawdgvfylhajhexcbex ikrcmgj hsyztaeoqyaczfbqoe  ygusulragaa nktxzsxiqowcqgywhkkpfyfqqwo
okvya fmdeuqvgwkthcwkasitwqtojlsolqpqodagmansgyjqxuctzjakaipczkzzucotfiuz twuyamoffhccsgtykhnzsdumxzuqgvbjbnartgytczcbftiktuyolgvwwmooni lfwdzfmlicctju p cxarif
Average loss at step 100: 5.318308 learning rate: 10.000000
Average loss at step 200: 4.611864 learning rate: 10.000000
Avera

# Appendix